# IBM Applied Data Science Capstone

This is a final project of a professional certification course "IBM Data Science"

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


#### Segmenting and Clustering Neighborhoods in Toronto: part 1

The process of an exploring neighborhoods in Toronto. The data has been parsed from [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [2]:
!conda install bs4

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |           py36_0         168 KB
    bs4-4.9.1                  |                0           4 KB
    ------------------------------------------------------------
                                           Total:         171 KB

The following NEW packages will be INSTALLED:

    bs4:             4.9.1-0                             

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                         --> 4.9.1-py36_0     
    ca-certificates: 2020.6.20-hecda079_0     conda-forge --> 2020.6.24-0      
    certifi:         2020.6.20-py36h9f0ad1d_0 conda-forge --> 2020.6.20-py36_0 

The following packages will be DOWNGRADED:

    openssl:         1.1.1g-

In [3]:
import requests
from bs4 import BeautifulSoup

In [62]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [79]:
r = requests.get(url)
soup = BeautifulSoup(r.text)
tables = soup.find_all('table')

In [80]:
len(tables)

3

In [81]:
table = tables[0]

df = pd.DataFrame(columns=range(0,3))
row_index = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    data = []
    for column in columns:
        data.append(column.get_text().rstrip())
    if(len(data) == 3):
        df.loc[row_index] = data
    row_index += 1
df.rename(columns={0: "PostalCode", 1: "Borough", 2: "Neighborhood"}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [82]:
df.shape

(180, 3)

In [83]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

In [84]:
df.shape

(103, 3)

#### Segmenting and Clustering Neighborhoods in Toronto: part 2

In [85]:
link_geo = 'https://cocl.us/Geospatial_data'
df_geo = pd.read_csv(link_geo)
df_geo = df_geo.set_index('Postal Code')
df_geo.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [86]:
df = df.set_index('PostalCode')

In [87]:
df = df.merge(df_geo, left_index=True, right_index=True)
df.head()

,Borough,Neighborhood,Latitude,Longitude
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [88]:
df.shape

(103, 4)